In [1]:
import torch
from torch.func import vmap, jvp, grad
from torch.optim import Adam

In [5]:
import torch
a = torch.arange(100).to(torch.float32)
torch.diff(a, dim=0)

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [2]:


def so3(phi):
    B = phi.shape[0]
    R = torch.zeros(B, 3, 3, device=phi.device, dtype=phi.dtype)
    R[:, 0, 1] = -phi[:, 2]
    R[:, 0, 2] = phi[:, 1]
    R[:, 1, 0] = phi[:, 2]
    R[:, 1, 2] = -phi[:, 0]
    R[:, 2, 0] = -phi[:, 1]
    R[:, 2, 1] = phi[:, 0]
    R = R + torch.eye(3, device=phi.device, dtype=phi.dtype).view(1, 3, 3)
    return R

def reproject(x, phi, t):
    
    R = so3(phi)
    R_inv = torch.linalg.inv(R)
    t_inv = -t
    xh = (x @ R_inv.transpose(-1, -2) + t_inv.view(4, 1, 3)) @ R.transpose(-1, -2) + t.view(4, 1, 3)
    return xh

pts = torch.randn(4, 10, 3).cuda()
phi = torch.randn(4, 3).cuda().requires_grad_()
t = torch.randn(4, 3).cuda().requires_grad_()
opt = Adam([phi, t], lr=1e-3)

for _ in range(1000):
    opt.zero_grad()
    xh = reproject(pts, phi, t)
    err = torch.norm(xh - pts, dim=-1).mean()
    err.backward()
    opt.step()